In [23]:
import random
import numpy as np
import simpy

# todo: config to be defined in config.py
conf = {
    'seed': None,
    'means': [25, 40, 20, 40], # mean times of all stages: next/prep/op/rec
    'rooms': [3, 1, 3], # totals of each identical facility: prep/op/rec
    'monitor_interval': 20 # snapshot interval for non-patient variables, such as queues
}

resu = {
    'patient_flow': [], # all flow times for each patient: arrival/prep/op/rec/leave
    'total_active': [0,0,0], # total time of prep/op/rec
    'util_active': [0,0,0], # active time of prep/op/rec
    'queues': [] # queue lengths for each facility, at snapshot time: time/prep/op/rec
}

In [22]:
# generator starts new patient process according to interval distribution
def patient_generator_mkB(env,conf,resu,facilities):
    while True:
        env.process(patient_mkB(env,conf,resu,facilities))
        yield env.timeout(random.expovariate(1/conf['means'][0]))

In [37]:
# individual patient, keeps track of actual service times for each stage: required time and extra waiting
def patient_mkB(env,conf,resu,facilities):
    # minimum processing times for each stage
    e = random.expovariate
    req_times = [e(1/conf['means'][1]),e(1/conf['means'][2]),e(1/conf['means'][3])]

    flow_times = [env.now, 0, 0, 0, 0] # arrival times at each stage, including release
    resu['patient_flow'].append(flow_times)
    
    # individual patient path through system
    prep = facilities[0].request()
    yield prep # wait prep room
    flow_times[1] = env.now # patient taken to a prep room
    
    yield env.timeout(req_times[0])
    op = facilities[1].request()
    yield op
    facilities[0].release(prep) # op room free, release prep
    flow_times[2] = env.now # patient enters operation
    
    yield env.timeout(req_times[1])
    rec = facilities[2].request()

    yield rec
    facilities[1].release(op) # op free once rec opens
    flow_times[3] = env.now # patient starts recovery

    yield env.timeout(req_times[2])
    facilities[2].release(rec) # rec done
    flow_times[4] = env.now # patient leaves

    # add to total stats
    resu['util_active'] += req_times
    resu['total_active'][0] += flow_times[2] - flow_times[1] # op start - prep start = prep total
    resu['total_active'][1] += flow_times[3] - flow_times[2]
    resu['total_active'][2] += flow_times[4] - flow_times[3]

In [38]:
def monitor_mkB(env,conf,resu,facilities):
    while True:
        resu['queues'].append([env.now, ])
        yield env.timeout(conf['monitor_interval'])

In [29]:
def create_hospital_simulation(env,conf):
    resu = {
        'patient_flow': [], # all flow times for each patient: arrival/prep/op/rec/leave
        'total_active': [0,0,0], # total time of prep/op/rec
        'util_active': [0,0,0] # active time of prep/op/rec
    }
    # seed the rng
    random.seed(a=conf['seed']) # doesnt need to check for None, seed() already does
    facilities = [
        simpy.Resource(env,capacity=conf['rooms'][0]),
        simpy.Resource(env,capacity=conf['rooms'][1]),
        simpy.Resource(env,capacity=conf['rooms'][2])
    ]
    env.process(monitor_mkB(env,conf,resu,facilities))
    env.process(patient_generator_mkB(env,conf,resu,facilities))
    return resu

In [32]:
env = simpy.Environment()
conf['seed'] = 2025
results = create_hospital_simulation(env,conf)

In [28]:
env.run(until=5)